# Performance, Portability and Productivity

##### Sections
- [Introduction to Performance, Portability and Productivity](#Introduction-to-Performance,-Portability-and-Productivity)
- [Introduction to oneAPI](#Introduction-to-oneAPI)
- [Test Application for Performance Portability](#Test-Application-for-Performance-Portability)
- [Analysis for Performance Portability](#Analysis-for-Performance-Portability)

## Learning Objectives
- Explain how the oneAPI programming model can solve the challenges of programming in a heterogeneous world.
- Understand the importance of writing Performance Portable code in heterogeneous world.
- Use tools like Intel VTune Profiler and Intel Advisor Roofline to performance analysis

## Introduction to Performance, Portability and Productivity

Developers of high performance computing applications are faced with an increasingly diverse number of computing platforms featuring multiple generations of CPUs, GPUs, FPGAs and ASICs. Developing code that is performant and portable across a diverse set of platforms is expensive in terms of time spent trying to achieve the best result across a given set of platforms.

This learning path will explore the use of oneAPI and Data Parallel C++ to demonstrate a method to achieve performant, portable code across five different platforms available on the Intel Devcloud. 

We will define Performance, Portability and Productivity as:

- __Performance__: Measurable quantity representing a characteristic of an application run on a platform. Typically, throughput or time-to-solution. We often express it as percentage of peak performance on platforms.

- __Portability__: Ability of application to run the correctly on different platforms.

- __Productivity__: The ratio of application code output to invested development effort, where application code output represents features, optimizations, maintenance, etc.


The achievement of performance, portability, and productivity builds on these three requirements: "An application is performance portable if it achieves a consistent ratio of the actual time-to-solution to either the best-known or the theoretical best time-to-solution on each platform with minimal platform-specific code required."

In order to demonstrate this, we explore several general matrix multiply (GEMM) algorithm examples using SYCL programming language, introduce several techniques to measure the effectiveness of the applications across the platforms, use timer functions inside the applications measuring kernel and compute times, and use this information to compute relative efficiency to a best implementation. In addition, we use roofline analysis and VTune™ analyzer to measure the application’s performance across the represented platforms.

Our learning path will not be an exhaustive optimization exercise nor considered benchmarking; rather, we focus on using SYCL as a method for heterogeneous programming that enables the developer to execute their code across CPUs and GPUs with minimal changes to the source. We also explore some techniques to improve the performance of the GEMM examples across platforms and introduce tools to gain insight into an application.


## Introduction to oneAPI

### Introducing oneAPI Programming Model
The __oneAPI__ programming model provides a comprehensive and unified portfolio of developer tools that can
be used across hardware targets, including a range of performance libraries spanning several workload
domains. The libraries include functions custom-coded for each target architecture so the same
function call delivers optimized performance across supported architectures. __SYCL__ is based on
industry standards and open specifications to encourage ecosystem collaboration and innovation.

### oneAPI Distribution
Intel&reg; oneAPI toolkits are available via multiple distribution channels:
* Local product installation: install the oneAPI toolkits from the __Intel® Developer Zone__.
* Install from containers or repositories: install the oneAPI toolkits from one of several supported
containers or repositories.
* Pre-installed in the __Intel® DevCloud__: a free development sandbox for access to the latest Intel® SVMS hardware and select oneAPI toolkits. 

### Programming Challenges for Multiple architectures
Currently in the data centric space there is growth in specialized workloads. Each kind of data centric hardware typically needs to be programmed using different languages and libraries as there is no common programming language or APIs, this requires maintaining separate code bases. Developers have to learn a whole set of different tools as there is inconsistent tool support across platforms. Developing software for each hardware platform requires a separate investment, with little ability to reuse that work to target a different architecture. You will also have to consider the requirement of the diverse set of data-centric hardware.

<img src="Assets/oneapi1.png">

### Introducing oneAPI
__oneAPI__ is a solution to deliver unified programming model to __simplify development__ across diverse architectures. It includes a unified and simplified language and libraries for expressing __parallelism__ and delivers uncompromised native high-level language performance across a range of hardware including __CPUs, GPUs, FPGAs__. oneAPI initiative is based on __industry standards and open specifications__ and is interoperable with existing HPC programming models.

<img src="Assets/oneapi2.png">

### What is Data Parallel C++ and SYCL
__Data Parallel C++ (DPC++)__ is oneAPI's implementation of SYCL. It takes advantage of modern C++ productivity benefits and familiar constructs, and incorporates the __SYCL*__ standard for data parallelism and heterogeneous programming. SYCL is a __single source__ language where host code and __heterogeneous accelerator kernels__ can be mixed in same source files. A SYCL program is invoked on the host computer and offloads the computation to an accelerator. Programmers use familiar C++ and library constructs with added functionalities like a __queue__ for work targeting, __buffer__ for data management, and __parallel_for__ for parallelism to direct which parts of the computation and data should be offloaded.

## Test Application for Performance Portability 

To demonstrate Performance Portability across CPUs and GPUs, we will implement a General __Matrix Multiplication__ (GEMM) in SYCL and work on tuning the code to achieve performance portability. We use Matrix Multiplication since it is very easy to understand and at the same time it is computationally intense especially when using large matrix sizes.

<img src="Assets/naive.PNG">

We will execute the GEMM code on the following four platforms and analyze all execution times, generate roofline report using Intel Advisor tool and also Profile the code using oneAPI Intel VTune Profiler.

### Platform Configuration 

- Intel® Xeon® E-2176G Processor with GEN9 GT2 Graphics 
- Intel (R) Core (TM) i9-10920X with Iris XE Max discrete Graphics
- Intel® Xeon® Gold 6128 Processor
- Intel® Xeon® Platinum 8153 Processor

### Code Structure

We will use SYCL programming language to implement Matrix Multiplication and offload to GPU or CPU.

In order to focus on code enhancements, we will architect our application by splitting the common code from the actual kernel implementation of matrix multiplication. 

Because the compute kernel is the only code that changes for various optimizations, we separated the __kernel computation logic code__ from the rest of the __common code__.

#### Kernel Code 

The actual kernel code for matrix multiplication implementation will be in a separate source file which is called from the common code source, we will be implementing various  approaches for the kernel code using SYCL and also use Math Kernel Library implementation.

We will be looking at various implementations of kernel code using different optimizations techniques and libraries in the next few modules

#### Common Code

The common code has implementation for command-line argument options, initializing the matrices, setting up the SYCL queue for offload, and a function to call the kernel code.

The common code is designed to input matrix dimension and work-group size if applicable, so that different matrix sizes can be set at runtime for computation and analysis using command-line arguments.

We calculate the __execution time__ of Matrix Multiplication kernel offloaded to device by collecting timestamps at the beginning and end of the kernel function.

##### Common Code walkthrough: - [src/mm_dpcpp_common.cpp](src/mm_dpcpp_common.cpp)

In [ ]:
%pycat src/mm_dpcpp_common.cpp

## Analysis for Performance Portability

We will use the kernel execution time computed for the various implementations to determine the Performance Portability.

We will also use two oneAPI analyzer tools for more detailed analysis:
- Intel Advisor Roofline
- Intel VTune Profiler

### Intel Advisor Roofline 

In addition to timing inside the application, we can use roofline analysis to better understand the application’s performance.

A roofline chart is a visual representation of application performance in relation to hardware limitations, including memory bandwidth and computational peaks. Intel® Advisor includes an automated roofline tool that measures and plots the chart on its own; all you need to do is read it.
The chart can be used to identify not only where bottlenecks exist, but also what’s likely causing them, and which ones will provide the most speed up if optimized.

Requirements for a roofline model on a GPU
To generate a roofline analysis report, an application must be at least partially running on a GPU, Gen9, or Gen11 integrated graphics card, and the offload must be implemented with OpenMP*, SYCL*, DPC++, or OpenCL™ software and a recent version of Intel Advisor.

Generating a roofline model on a GPU generates a multilevel roofline, where a single loop generates several dots, and each dot can be compared to its own memory (GTI/L3/DRAM/SLM).

To run a roofline analysis, perform the following three steps. These are the same steps one would use on any application.


#### Step 1: Survey Device
advisor --collect=survey --static-instruction-mix --stackwalk-mode=online --static-instruction-mix --project-dir=./roofline_data -- lab/mm_dpcpp_basic -n 1024 -m 16

#### Step 2: Run Trip Counts and FLOP Analysis
advisor --collect=tripcounts --project-dir=./advisor_gen9 --flop --ignore-checksums --stacks --track-memory-objects --track-stack-accesses --enable-data-transfer-analysis --profile-jit --cache-sources --enable-cache-simulation -- lab/mm_dpcpp_basic -n 1024 -m 16

#### Step 3: Create Roofline Report
advisor --report=roofline --gpu --project-dir=./roofline_data --report-output=./roofline_gpu_gen9_1024_mm_dpcpp_basic.html

The report generated is live and supports zooming and the ability to configure levels to display.  Make sure to check all the levels as you will see the difference between platforms.

<img src="Assets/r1.png">

More details for Intel Advisor Roofline Model can be found [here](https://www.intel.com/content/www/us/en/developer/articles/guide/intel-advisor-roofline.html)

Below is helper script we use to generate Roofline report, we update the script with binary name, matrix size and other options and run on the desired node:

In [ ]:
%pycat roofline_all.sh

### Intel VTune™ Profiler

For additional insight, we also profiled the application using VTune analyzer, and looked for GPU hotspots in the code.

VTune analyzer enables profiling capabilities so you can tune for CPU, GPU, and FPGA devices.

Because we are executing without a GUI, we will use the VTune analyzer's command-line interface, and generate an HTML report to see the results. We look for hotspots to better understand where the bottlenecks in the application manifested.


#### Step 1: Run and collect VTune Profiler Data
```vtune -collect gpu_hotspots -result-dir vtune_data a.out```

Various types of profiling data can be collected like `hotspots`, `memory-consumption`, `memory-access`, `threading`…

Use the command line help to find out more: ```vtune --help -collect```

#### Step 2: Generate HTML Report for Collected VTune Profiler data:
```vtune -report summary -result-dir vtune_data -format html -report-output $(pwd)/summary.html```

Various types of report can be generated like `summary`, `top-down`, `callstacks`…

Use the command line help to find out more:  ```vtune --help -report```

### VTune Results

The VTune results will show the following characteristics 

##### GPU L3 Bound

This metric shows how often the GPU was idle or stalled on the L3 cache.
Possible Issues: L3 bandwidth was high when EUs were stalled or idle. Consider improving cache reuse.


##### EU Stalled/Idle

The average time the EUs were stalled or idle.
Possible Issues: The time when the EUs were stalled or idle is high, which has a negative impact on compute-bound applications.


##### FPU Utilization

This metric represents how intensively your program uses the FPU. 100 percent means that the FPU is fully loaded and is retiring a vector instruction with full capacity every cycle of the application execution.
Possible Issues: The metric value is low. This can indicate poor FPU utilization because of non-vectorized floating point operations, or inefficient vectorization due to legacy vector instruction set or memory access pattern issues. Consider using vector analysis in Intel Advisor for data and tips to improve vectorization efficiency in your application.


##### Occupancy

The normalized sum of all cycles on all core and thread slots when a slot has a thread scheduled. Possible Issues: Low value of the occupancy metric may be caused by inefficient work scheduling. Make sure work items are neither too small nor too large.

##### Deep Dive

To really dig into the results, the best method is to move the VTune analyzer collection data to a laptop that has the VTune analyzer GUI installed, and explore from there.

More details for Intel VTune Profile can be found [here](https://www.intel.com/content/www/us/en/develop/documentation/vtune-help/top/analyze-performance.html)

Below is helper script we use to generate VTune report, we update the script with binary name, matrix size and other options and run on the desired node:

In [ ]:
%pycat vtune_all.sh

### Summary

In this module we introduced SYCL a heterogeneous programming language where host code and heterogeneous accelerator kernels can be mixed in same source files. A SYCL program is invoked on the host computer and offloads the computation to a chosen accelerator. By using an available library example as our baseline we established the performance gaol across multiple platforms.  

An application is performance portable if it achieves a consistent ratio of the actual time to solution to either the best-known or the theoretical best time to solution on each platform with minimal platform specific code required. However, there are not always libraries for every function and libraries have their own set of challenges with respect to support across different versions of software and hardware.

In the next module we will explore how one could go about achieving library like performance while writing as little code as possible.  We will begin with a basic GEMM and enhance the code using SYCL features.  Our goal is to demonstrate how small changes to the code can enable portable performance.



Sources:  
    1. https://performanceportability.org/perfport/definition/

## Resources

Check out these related resources

#### Intel® oneAPI Toolkit documentation
* [Intel® oneAPI main page](https://software.intel.com/oneapi "oneAPI main page")
* [Intel® oneAPI programming guide](https://software.intel.com/sites/default/files/oneAPIProgrammingGuide_3.pdf "oneAPI programming guide")
* [Intel® DevCloud Signup](https://software.intel.com/en-us/devcloud/oneapi "Intel DevCloud")  Sign up here if you do not have an account.
* [Intel® DevCloud Connect](https://devcloud.intel.com/oneapi/get_started/)  Login to the DevCloud here.
* [Get Started with oneAPI for Linux*](https://software.intel.com/en-us/get-started-with-intel-oneapi-linux)
* [Get Started with oneAPI for Windows*](https://software.intel.com/en-us/get-started-with-intel-oneapi-windows)
* [Intel® oneAPI Code Samples](https://software.intel.com/en-us/articles/code-samples-for-intel-oneapibeta-toolkits)
* [oneAPI Specification elements](https://www.oneapi.com/spec/)

#### SYCL 
* [SYCL* 2020 Specification](https://www.khronos.org/registry/SYCL/specs/sycl-2020/pdf/sycl-2020.pdf)

#### Modern C++
* [CPPReference](https://en.cppreference.com/w/)
* [CPlusPlus](http://www.cplusplus.com/)

***

#### Notices

Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure.
Your costs and results may vary.
© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.
